In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
import locale
import sys
import os
import re

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

# Leyendo lista de Colegios y Estudiantes

In [4]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [5]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [6]:
estuDic

{'20061': 'SB11_20061.txt',
 '20062': 'SB11_20062.txt',
 '20071': 'SB11_20071.txt',
 '20072': 'SB11_20072.txt',
 '20081': 'SB11_20081.txt',
 '20082': 'SB11_20082.txt',
 '20091': 'SB11_20091.txt',
 '20092': 'SB11_20092.txt',
 '20101': 'SB11_20101.txt',
 '20102': 'SB11_20102.txt',
 '20111': 'SB11_20111.txt',
 '20112': 'SB11_20112.txt',
 '20121': 'SB11_20121.txt',
 '20122': 'SB11_20122.txt',
 '20131': 'SB11_20131.txt',
 '20132': 'SB11_20132.txt',
 '20141': 'SB11_20141.txt',
 '20142': 'SB11_20142.txt',
 '20151': 'SB11_20151.txt',
 '20152': 'SB11_20152.txt',
 '20161': 'SB11_20161.txt',
 '20162': 'SB11_20162.txt',
 '20171': 'SB11_20171.txt',
 '20172': 'SB11_20172.txt',
 '20181': 'SB11_20181.txt',
 '20182': 'SB11_20182.txt',
 '20191': 'SB11_20191.txt'}

In [7]:
coleDic

{'2006': 'SB11-CLASIFI-PLANTELES-2006.txt',
 '2007': 'SB11-CLASIFI-PLANTELES-2007.txt',
 '2008': 'SB11-CLASIFI-PLANTELES-2008.txt',
 '2009': 'SB11-CLASIFI-PLANTELES-2009.txt',
 '2010': 'SB11-CLASIFI-PLANTELES-2010.txt',
 '2011': 'SB11-CLASIFI-PLANTELES-2011.txt',
 '2012': 'SB11-CLASIFI-PLANTELES-2012.txt',
 '2013': 'SB11-CLASIFI-PLANTELES-2013.txt',
 '20141': 'SB11-CLASIFI-PLANTELES-20141.txt',
 '20142': 'SB11-CLASIFI-PLANTELES-20142.txt',
 '20151': 'SB11-CLASIFI-PLANTELES-20151.txt',
 '20152': 'SB11-CLASIFI-PLANTELES-20152.txt',
 '20161': 'SB11-CLASIFI-PLANTELES-20161.txt',
 '20162': 'SB11-CLASIFI-PLANTELES-20162.txt',
 '20171': 'SB11-CLASIFI-PLANTELES-20171.csv',
 '20172': 'SB11-CLASIFI-PLANTELES-20172.csv',
 '20181': 'SB11-CLASIFI-PLANTELES-20181.txt',
 '20182': 'SB11-CLASIFI-PLANTELES-20182.txt'}

# Analizando estructura de Colegios

In [8]:
for colegio in colegios:
    lines = sc.textFile(base + '/' + colegio)
    print("Archivo: " + colegio + "\n" + lines.first() + '\n')

Archivo: SB11-CLASIFI-PLANTELES-2006.txt
COLE_CODIGO_COLEGIO	COLE_INST_NOMBRE	COLE_CODMPIO_COLEGIO	COLE_MPIO_COLEGIO	COLE_DEPTO_COLEGIO	COLE_INST_JORNADA	COLE_CALENDARIO_COLEGIO	COLE_GENEROPOBLACION	COLE_NATURALEZA	COLE_CIENCIAS_SOCIALES	COLE_QUIMICA	COLE_FISICA	COLE_BIOLOGIA	COLE_FILOSOFIA	COLE_MATEMATICAS	COLE_LENGUAJE	COLE_INGLES	COLE_GEOGRAFIA	COLE_HISTORIA	COLE_CATEGORIA	COLE_ESTUDIANTES_PRESENTES

Archivo: SB11-CLASIFI-PLANTELES-2007.txt
COLE_CODIGO_COLEGIO	COLE_INST_NOMBRE	COLE_CODMPIO_COLEGIO	COLE_MPIO_COLEGIO	COLE_DEPTO_COLEGIO	COLE_INST_JORNADA	COLE_CALENDARIO_COLEGIO	COLE_GENEROPOBLACION	COLE_NATURALEZA	COLE_CIENCIAS_SOCIALES	COLE_QUIMICA	COLE_FISICA	COLE_BIOLOGIA	COLE_FILOSOFIA	COLE_MATEMATICAS	COLE_LENGUAJE	COLE_INGLES	COLE_GEOGRAFIA	COLE_HISTORIA	COLE_CATEGORIA	COLE_ESTUDIANTES_PRESENTES

Archivo: SB11-CLASIFI-PLANTELES-2008.txt
COLE_CODIGO_COLEGIO	COLE_INST_NOMBRE	COLE_CODMPIO_COLEGIO	COLE_MPIO_COLEGIO	COLE_DEPTO_COLEGIO	COLE_INST_JORNADA	COLE_CALENDARIO_COLEGIO	COLE_GEN

# Analizando estructura de estudiantes

In [9]:
for estu in estudiantes:
    linea = sc.textFile(base2 + '/' + estu)
    print("Archivo: " + estu + "\n" + linea.first() + '\n')

Archivo: SB11_20061.txt
estu_exam_nombreexamen¬estu_estudiante¬periodo¬estu_consecutivo¬estu_edad¬estu_tipo_documento¬estu_pais_reside¬estu_genero¬estu_nacimiento_dia¬estu_nacimiento_mes¬estu_nacimiento_anno¬estu_cod_reside_mcpio¬estu_reside_mcpio¬estu_reside_depto¬estu_zona_reside¬estu_razoninstituto¬cole_valor_pension¬estu_ies_cod_deseada¬estu_carrdeseada_razon¬estu_carrdeseada_cod¬estu_ies_deseada_nombre¬estu_ies_cod_mpio_deseada¬estu_ies_mpio_deseada¬estu_ies_dept_deseada¬estu_mpio_presentacion¬estu_dept_presentacion¬estu_exam_cod_mpio_presentacio¬estu_ano_termino_bachill¬cole_cod_icfes¬cole_cod_dane_institucion¬cole_nombre_sede¬cole_cod_mcpio_ubicacion¬cole_calendario¬cole_genero¬cole_naturaleza¬cole_bilingue¬cole_jornada¬cole_caracter¬punt_lenguaje¬punt_matematicas¬punt_c_sociales¬punt_filosofia¬punt_biologia¬punt_quimica¬punt_fisica¬nombre_idioma¬punt_idioma¬nombre_profundizacion¬punt_profundizacion¬desemp_profundizacion¬nombre_interdisciplinar¬punt_interdisciplinar¬estu_puesto


Archivo: SB11_20112.txt
ESTU_TIPODOCUMENTO¬ESTU_NACIONALIDAD¬ESTU_GENERO¬ESTU_FECHANACIMIENTO¬PERIODO¬ESTU_CONSECUTIVO¬ESTU_ESTUDIANTE¬ESTU_PAIS_RESIDE¬ESTU_DEPTO_RESIDE¬ESTU_COD_RESIDE_DEPTO¬ESTU_MCPIO_RESIDE¬ESTU_COD_RESIDE_MCPIO¬ESTU_AREARESIDE¬ESTU_MESTERMINOBACHILLER¬ESTU_ANOTERMINOBACHILLER¬ESTU_VALORPENSIONCOLEGIO¬ESTU_MOTIVO_PRESENTAVALIDACION¬ESTU_VECESPRESENTOEXAMEN¬ESTU_FECHAGRADOBACHILLER¬FAMI_EDUCACIONPADRE¬FAMI_EDUCACIONMADRE¬FAMI_OCUPACIONPADRE¬FAMI_OCUPACIONMADRE¬FAMI_ESTRATOVIVIENDA¬FAMI_NIVELSISBEN¬FAMI_PERSONASHOGAR¬FAMI_CUARTOSHOGAR¬FAMI_PISOSHOGAR¬FAMI_TIENEINTERNET¬FAMI_TIENE_SERVICIOTV¬FAMI_TIENECOMPUTADOR¬FAMI_TIENELAVADORA¬FAMI_TIENEMICROONDAS¬FAMI_TIENEHORNO¬FAMI_TIENEAUTOMOVIL¬FAMI_TIENEDVD¬FAMI_TIENE_NEVERA¬FAMI_TIENE_CELULAR¬FAMI_TELEFONO¬FAMI_INGRESOFMILIARMENSUAL¬ESTU_TRABAJAACTUALMENTE¬ESTU_ANTECEDENTES¬ESTU_ANOSPREESCOLAR¬ESTU_ANOMATRICULAPRIMERO¬ESTU_ANOTERMINOQUINTO¬ESTU_ANOMATRICULASEXTO¬ESTU_REPROBOPRIMERO¬ESTU_REPROBOSEGUNDO¬ESTU_REPROBOTERCERO¬EST

Archivo: SB11_20151.txt
ESTU_TIPODOCUMENTO¬ESTU_NACIONALIDAD¬ESTU_GENERO¬ESTU_FECHANACIMIENTO¬PERIODO¬ESTU_CONSECUTIVO¬ESTU_ESTUDIANTE¬ESTU_PAIS_RESIDE¬ESTU_ETNIA¬ESTU_LIMITA_MOTRIZ¬ESTU_LIMITA_INVIDENTE¬ESTU_LIMITA_CONDICIONESPECIAL¬ESTU_LIMITA_SORDO¬ESTU_LIMITA_SDOWN¬ESTU_LIMITA_AUTISMO¬ESTU_DEPTO_RESIDE¬ESTU_COD_RESIDE_DEPTO¬ESTU_MCPIO_RESIDE¬ESTU_COD_RESIDE_MCPIO¬ESTU_AREARESIDE¬ESTU_VALORPENSIONCOLEGIO¬ESTU_VECESPRESENTOEXAMEN¬FAMI_EDUCACIONPADRE¬FAMI_EDUCACIONMADRE¬FAMI_OCUPACIONPADRE¬FAMI_OCUPACIONMADRE¬FAMI_NUMHERMANOS¬FAMI_ESTRATOVIVIENDA¬FAMI_NIVELSISBEN¬FAMI_PERSONASHOGAR¬FAMI_CUARTOSHOGAR¬FAMI_PISOSHOGAR¬FAMI_TIENEINTERNET¬FAMI_TIENECOMPUTADOR¬FAMI_TIENELAVADORA¬FAMI_TIENEMICROONDAS¬FAMI_TIENEHORNO¬FAMI_TIENESERVICIOTV¬FAMI_TIENEAUTOMOVIL¬FAMI_TIENEDVD¬FAMI_TELEFONO¬FAMI_NUMLIBROS¬FAMI_INGRESOFMILIARMENSUAL¬ESTU_TRABAJAACTUALMENTE¬ESTU_RECIBESALARIO¬ESTU_ANTECEDENTES¬ESTU_ANOSPREESCOLAR¬ESTU_ANOMATRICULAPRIMERO¬ESTU_ANOTERMINOQUINTO¬ESTU_ANOMATRICULASEXTO¬ESTU_REPROBOPRIMER

Archivo: SB11_20191.txt
ESTU_TIPODOCUMENTO¬ESTU_NACIONALIDAD¬ESTU_GENERO¬ESTU_FECHANACIMIENTO¬PERIODO¬ESTU_CONSECUTIVO¬ESTU_ESTUDIANTE¬ESTU_PAIS_RESIDE¬ESTU_TIENEETNIA¬ESTU_ETNIA¬ESTU_DEPTO_RESIDE¬ESTU_COD_RESIDE_DEPTO¬ESTU_MCPIO_RESIDE¬ESTU_COD_RESIDE_MCPIO¬FAMI_ESTRATOVIVIENDA¬FAMI_PERSONASHOGAR¬FAMI_CUARTOSHOGAR¬FAMI_EDUCACIONPADRE¬FAMI_EDUCACIONMADRE¬FAMI_TRABAJOLABORPADRE¬FAMI_TRABAJOLABORMADRE¬FAMI_TIENEINTERNET¬FAMI_TIENESERVICIOTV¬FAMI_TIENECOMPUTADOR¬FAMI_TIENELAVADORA¬FAMI_TIENEHORNOMICROOGAS¬FAMI_TIENEAUTOMOVIL¬FAMI_TIENEMOTOCICLETA¬FAMI_TIENECONSOLAVIDEOJUEGOS¬FAMI_NUMLIBROS¬FAMI_COMELECHEDERIVADOS¬FAMI_COMECARNEPESCADOHUEVO¬FAMI_COMECEREALFRUTOSLEGUMBRE¬FAMI_SITUACIONECONOMICA¬ESTU_DEDICACIONLECTURADIARIA¬ESTU_DEDICACIONINTERNET¬ESTU_HORASSEMANATRABAJA¬ESTU_TIPOREMUNERACION¬COLE_CODIGO_ICFES¬COLE_COD_DANE_ESTABLECIMIENTO¬COLE_NOMBRE_ESTABLECIMIENTO¬COLE_GENERO¬COLE_NATURALEZA¬COLE_CALENDARIO¬COLE_BILINGUE¬COLE_CARACTER¬COLE_COD_DANE_SEDE¬COLE_NOMBRE_SEDE¬COLE_SEDE_PRINCIPA

In [10]:
dfestu = {}
dfcole = {}
for i in coleDic:
    dfestu[i] = None
    dfcole[i] = None

# 2006

In [11]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [12]:
print("Archivos de estudiantes: ", estuDic['20061'],estuDic['20062'])
print("Archivos de colegios: ",coleDic['2006'])

Archivos de estudiantes:  SB11_20061.txt SB11_20062.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-2006.txt


In [13]:
# SE leen los dos archivos y se crean dos dataframes
df1 = spark.read.load(base2 + '/' + estuDic['20061'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')
df2 = spark.read.load(base2 + '/' + estuDic['20062'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [15]:
columnasPuntos = [
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_idioma',
 'punt_profundizacion',
 'punt_interdisciplinar']

In [16]:
# SE unen los dos dataframes
df = unionAll(df1,df2)
for cp in columnasPuntos:
    df = df.withColumn(cp, regexp_replace(cp, ',', '.'))
    df = df.withColumn(cp, df[cp].cast("float"))
    
df1 = df2 = None

In [63]:
print("Numero de filas ", df.count())

Numero de filas  471878


In [17]:
print(df.printSchema())

root
 |-- estu_exam_nombreexamen: string (nullable = true)
 |-- estu_estudiante: integer (nullable = true)
 |-- periodo: integer (nullable = true)
 |-- estu_consecutivo: string (nullable = true)
 |-- estu_edad: integer (nullable = true)
 |-- estu_tipo_documento: string (nullable = true)
 |-- estu_pais_reside: string (nullable = true)
 |-- estu_genero: string (nullable = true)
 |-- estu_nacimiento_dia: integer (nullable = true)
 |-- estu_nacimiento_mes: integer (nullable = true)
 |-- estu_nacimiento_anno: integer (nullable = true)
 |-- estu_cod_reside_mcpio: integer (nullable = true)
 |-- estu_reside_mcpio: string (nullable = true)
 |-- estu_reside_depto: string (nullable = true)
 |-- estu_zona_reside: integer (nullable = true)
 |-- estu_razoninstituto: integer (nullable = true)
 |-- cole_valor_pension: integer (nullable = true)
 |-- estu_ies_cod_deseada: integer (nullable = true)
 |-- estu_carrdeseada_razon: integer (nullable = true)
 |-- estu_carrdeseada_cod: integer (nullable = true)

In [18]:
df.columns

['estu_exam_nombreexamen',
 'estu_estudiante',
 'periodo',
 'estu_consecutivo',
 'estu_edad',
 'estu_tipo_documento',
 'estu_pais_reside',
 'estu_genero',
 'estu_nacimiento_dia',
 'estu_nacimiento_mes',
 'estu_nacimiento_anno',
 'estu_cod_reside_mcpio',
 'estu_reside_mcpio',
 'estu_reside_depto',
 'estu_zona_reside',
 'estu_razoninstituto',
 'cole_valor_pension',
 'estu_ies_cod_deseada',
 'estu_carrdeseada_razon',
 'estu_carrdeseada_cod',
 'estu_ies_deseada_nombre',
 'estu_ies_cod_mpio_deseada',
 'estu_ies_mpio_deseada',
 'estu_ies_dept_deseada',
 'estu_mpio_presentacion',
 'estu_dept_presentacion',
 'estu_exam_cod_mpio_presentacio',
 'estu_ano_termino_bachill',
 'cole_cod_icfes',
 'cole_cod_dane_institucion',
 'cole_nombre_sede',
 'cole_cod_mcpio_ubicacion',
 'cole_calendario',
 'cole_genero',
 'cole_naturaleza',
 'cole_bilingue',
 'cole_jornada',
 'cole_caracter',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_f

In [19]:
# Seleccion de todas las instituciones diferentes
dsIns_dane = df.select('cole_cod_dane_institucion').distinct()

In [20]:
dsIns_dane.count()

7750

In [21]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst = df.groupBy('cole_cod_dane_institucion').count().orderBy('count',ascending=
False)

In [22]:
dsCountEst.show(10,False)

+-------------------------+-----+
|cole_cod_dane_institucion|count|
+-------------------------+-----+
|null                     |7531 |
|105001000108             |1200 |
|111001024732             |987  |
|376001001132             |843  |
|168001003591             |817  |
|111001019411             |811  |
|105001013340             |798  |
|113001003274             |702  |
|108758000031             |632  |
|305440000994             |627  |
+-------------------------+-----+
only showing top 10 rows



In [23]:
#Se crea una columna con el total de los puntajes   
df= df.withColumn("Suma",df['punt_lenguaje']+df['punt_matematicas']+df['punt_c_sociales']+
                     df['punt_filosofia']+df['punt_biologia']+df['punt_quimica']+df['punt_fisica']+
                     df['punt_idioma']+df['punt_profundizacion']+df['punt_interdisciplinar'])

In [25]:
#Se crea una lista de columnas para hacer el analisis.
columnasToAnalysis = columnasPuntos.copy()
columnasToAnalysis.insert(0,'cole_cod_dane_institucion')
columnasToAnalysis.append('Suma')

In [26]:
columnasToAnalysis

['cole_cod_dane_institucion',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_idioma',
 'punt_profundizacion',
 'punt_interdisciplinar',
 'Suma']

In [28]:
# SE Selecciona el subgrupo de informciòn que se analizara
df2006 = df.select(columnasToAnalysis)

In [29]:
df2006.columns

['cole_cod_dane_institucion',
 'punt_lenguaje',
 'punt_matematicas',
 'punt_c_sociales',
 'punt_filosofia',
 'punt_biologia',
 'punt_quimica',
 'punt_fisica',
 'punt_idioma',
 'punt_profundizacion',
 'punt_interdisciplinar',
 'Suma']

## Resultados finales por instituciòn

In [32]:
# Còdigo para obtener la media de los resultados de las evaluaciones de todos los estudiantes por instituciòn 
# y posiciòn en orden descendente.
from pyspark.sql.functions import *
from pyspark.sql.window import Window

df2006p = df2006.groupBy('cole_cod_dane_institucion').mean()
df2006p = df2006p.withColumn("rank", dense_rank().over(Window.orderBy(desc("avg(Suma)"))))
df2006p = df2006p.orderBy('rank',ascending= True)

In [33]:
df2006p.show(10,False)

+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+------------------+-----------------+------------------------+--------------------------+-----------------+----+
|cole_cod_dane_institucion|avg(punt_lenguaje)|avg(punt_matematicas)|avg(punt_c_sociales)|avg(punt_filosofia)|avg(punt_biologia)|avg(punt_quimica) |avg(punt_fisica)  |avg(punt_idioma) |avg(punt_profundizacion)|avg(punt_interdisciplinar)|avg(Suma)        |rank|
+-------------------------+------------------+---------------------+--------------------+-------------------+------------------+------------------+------------------+-----------------+------------------------+--------------------------+-----------------+----+
|311001089221             |65.57875061035156 |65.78999948501587    |69.42500019073486   |61.70125079154968  |69.47937536239624 |61.438124656677246|62.29437446594238 |70.63875007629395|6.42750009894371        |57.77499985

## Colegios 2006

In [ ]:
# se crea un dataframe con una columna que lo señala el año 2006.
from pyspark.sql.functions import lit
df = df2006p.withColumn('año',lit(2006))
df.show(5,False)

In [35]:
# SE crea Dataframe de colegios 2006
dfcol2006 = spark.read.load(base + '/' + coleDic['2006'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='\t')
dfcol2006 = dfcol2006.withColumn('COLE_CODIGO_COLEGIO', dfcol2006['COLE_CODIGO_COLEGIO'].cast("string"))
dfcol2006 = dfcol2006.withColumn('COLE_CODMPIO_COLEGIO', dfcol2006['COLE_CODMPIO_COLEGIO'].cast("string"))

In [36]:
print(dfcol2006.printSchema())

root
 |-- COLE_CODIGO_COLEGIO: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: string (nullable = true)
 |-- COLE_MPIO_COLEGIO: string (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_INST_JORNADA: string (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_CIENCIAS_SOCIALES: integer (nullable = true)
 |-- COLE_QUIMICA: integer (nullable = true)
 |-- COLE_FISICA: integer (nullable = true)
 |-- COLE_BIOLOGIA: integer (nullable = true)
 |-- COLE_FILOSOFIA: integer (nullable = true)
 |-- COLE_MATEMATICAS: integer (nullable = true)
 |-- COLE_LENGUAJE: integer (nullable = true)
 |-- COLE_INGLES: string (nullable = true)
 |-- COLE_GEOGRAFIA: string (nullable = true)
 |-- COLE_HISTORIA: string (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- COLE_ESTUDIANTES_PRESENTE

In [37]:
dfcol2006.columns

['COLE_CODIGO_COLEGIO',
 'COLE_INST_NOMBRE',
 'COLE_CODMPIO_COLEGIO',
 'COLE_MPIO_COLEGIO',
 'COLE_DEPTO_COLEGIO',
 'COLE_INST_JORNADA',
 'COLE_CALENDARIO_COLEGIO',
 'COLE_GENEROPOBLACION',
 'COLE_NATURALEZA',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES',
 'COLE_GEOGRAFIA',
 'COLE_HISTORIA',
 'COLE_CATEGORIA',
 'COLE_ESTUDIANTES_PRESENTES']

In [38]:
colsColegio = ['COLE_CODIGO_COLEGIO',
 'COLE_CODMPIO_COLEGIO',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES',
 'COLE_GEOGRAFIA',
 'COLE_HISTORIA']

In [39]:
dfcol2006 = dfcol2006.select(colsColegio)

In [40]:
dfcol2006.show(5,False)

+-------------------+--------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|COLE_CODIGO_COLEGIO|COLE_CODMPIO_COLEGIO|COLE_CIENCIAS_SOCIALES|COLE_QUIMICA|COLE_FISICA|COLE_BIOLOGIA|COLE_FILOSOFIA|COLE_MATEMATICAS|COLE_LENGUAJE|COLE_INGLES|COLE_GEOGRAFIA|COLE_HISTORIA|
+-------------------+--------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|127696             |5001                |6                     |6           |7          |6            |8             |6               |7            |null       |null          |null         |
|127704             |20001               |7                     |7           |7          |7            |8             |6               |7            |null       |null          |null         |
|127712             |41807              

In [43]:
print("El numero de colegios año 2006 es: ", dfcol2006.select('COLE_CODIGO_COLEGIO').distinct().count())

El numero de colegios año 2006 es:  9249


## Anàlisis comparativo entre estudiantes y colegios año 2006

In [46]:
dfjoin2006 = df.join(dfcol2006, df.cole_cod_dane_institucion == dfcol2006.COLE_CODIGO_COLEGIO)

In [47]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin2006.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


In [48]:
dfjoin2006 = df.join(dfcol2006, df.cole_cod_dane_institucion == dfcol2006.COLE_CODMPIO_COLEGIO)
dfjoin2006.count()
print("Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios:", dfjoin2006.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes y colegios: 0


La informaciòn de colegios no coincide con la informaciòn de estudiantes, no existe cole_cod_dane_institucion en el archivo de colegios.